In [1]:
import numpy as np
from qutip import *
from sympy.physics.quantum.cg import CG
from sympy.physics.wigner import wigner_6j, wigner_3j
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.linalg import inv
from numpy import linalg as LA
from scipy.optimize import minimize_scalar
import math
from scipy.optimize import curve_fit
from scipy.sparse.linalg import eigs
from scipy.odr import ODR, Model, Data, RealData
import matplotlib.cm as cm

In [2]:
I = 1
S = 1
Je = 0
Jg = 1
Lg = 2
F = [i for i in range(int(I-Jg),int(I + Jg + 1),1)]
F.insert(0,int(I+Je))

Natomic = [2*F[i] + 1 for i in range(len(F))]
Natomic = sum(Natomic)

Nmotion = 15
a = tensor(qeye(Natomic),destroy(Nmotion))
y = a+a.dag()
     
#system parameters
c = 299792458
λ = 646e-9
f = c/λ 
f_1 = 11.2e9
f_2 = 10.5e9

#176Lu+
amu = 1.66053886e-27
hbar = 1.054571817e-34
M = 176*amu

omega0  = 1.4

eta = []

eta2 = 2*np.pi*f/c*np.sqrt(hbar/(2*M*2*np.pi*omega0*10**(6)))
eta.append(eta2)

eta3 = 2*np.pi*(f-f_1)/c*np.sqrt(hbar/(2*M*2*np.pi*omega0*10**(6)))
eta.append(eta3)

eta4 = -2*np.pi*(f-f_2)/c*np.sqrt(hbar/(2*M*2*np.pi*omega0*10**(6)))
eta.append(eta4)


II = 7
S = 1
Je = 0
Jg = 1
Lg = 2
FF = [i for i in range(int(II-Jg),int(II + Jg + 1),1)]
FF.insert(0,int(II+Je))

#Zeeman stuff


muB = 9.3e-24 #Bohr magneton J T^-1
hbar = 1.054e-34 # J s

muB = muB/hbar #hbar = 1
muB = muB/(2*np.pi)
muB = muB/1e6  # so it's in 1/2π (MHz T^-1)

# 1Gauss is 0.0001 T
B = 5e-4

#Lande g factor for F
me = 9.1093837015e-31 
mp = 1.67262192369e-27 

mI = 3.169 #nuclear magnetic moment of 176Lu in units of μN
gI = mI/II

gJg = 1 + (Jg*(Jg +1) + S*(S + 1) - Lg*(Lg + 1))/(2*Jg*(Jg + 1))
#print(gJg)

#gF = [1,0,1,2]
gF=[]
gF.append(-gI*me/mp)

for i in range(3):
    gF.append(gJg*(FF[i +1]*(FF[i+1] + 1) + Jg*(Jg+1) - II*(II+1))/(2*FF[i+1]*(FF[i +1] + 1)))

def base(F,i,mF):
    #numbering states from 0 to N-1 starting from -mF to mF
    # 0 is |F',-mF'>
    if i==0:
        b = tensor(basis(Natomic,mF+F[i], dtype="csr"),qeye(Nmotion))
    elif i==1:
        b = tensor(basis(Natomic,mF + F[i] + 2*F[0] + 1, dtype="csr"),qeye(Nmotion))
    elif i==2:
        b = tensor(basis(Natomic,mF + F[i] + 2*F[0] + 1 + 2*F[1] + 1, dtype="csr"),qeye(Nmotion))
    else:
        b = tensor(basis(Natomic,mF + F[i] + 2*F[0] + 1 + 2*F[1] + 1 + 2*F[2] +1, dtype="csr"),qeye(Nmotion))
    return b   


def GammaFgFe(F,ig,Je,Jg,I,GammaJgJe):
    return float((2*Je + 1)*(2*F[0] + 1)*wigner_6j(Je,F[0],I,F[ig],Jg,1)**2)*2*np.pi*GammaJgJe
    #return GammaJgJe

def cg(F,ig,mFg,ie,mFe,q):
    return float(CG(F[ig],mFg,1,q,F[ie],mFe).doit())
    #return 1



#Hamiltonian
def H_I(F,Omega_p):
    HI=0*tensor(basis(Natomic,0, dtype="csr"),qeye(Nmotion))*tensor(basis(Natomic,0, dtype="csr"),qeye(Nmotion)).dag()
    for mFe in range(-F[0],F[0] + 1):
        for ig in range(1,len(F)):
            for mFg in range(-F[ig],F[ig]+1):
                for q in range(-1,2):
                    #HI += 2*np.pi*cg(F,ig,mFg,0,mFe,q)*Omega_p[ig-1,q+1]/2*(-1j*eta[ig-1]*y).expm(dtype="csr")*base(F,0,mFe)*base(F,ig,mFg).dag()
                    HI += 2*np.pi*cg(F,ig,mFg,0,mFe,q)*Omega_p[ig-1,q+1]/2*base(F,0,mFe)*base(F,ig,mFg).dag()*(tensor(qeye(Natomic),qeye(Nmotion)) +1j*eta[ig-1]*y)
                
    return -1/np.sqrt(2*F[0]+1)*(HI + HI.dag()) 

def H_0(F,Delta):
    H0 = 0*tensor(basis(Natomic,0, dtype="csr"),qeye(Nmotion))*tensor(basis(Natomic,0, dtype="csr"),qeye(Nmotion)).dag()
    for l in range(len(F)):
        for mF in range(-F[l],F[l]+1):
            H0 += 2*np.pi*(Delta[l] + gF[l]*muB*mF*B)*base(F,l,mF)*base(F,l,mF).dag() 
    return H0

In [3]:
#Setting up


#Rabi frequencies Ω_F,F' #1/2π (MHz)
omega = [23.5,10,23.5]

#turn on/off polarization
Omega_p= np.zeros((3,3)) # (ig-1, q + 1) transition from F state to F' with polarization q

Omega_p[0,0] = Omega_p[0,2] = omega[0]       #σ+ and σ- for F=F[1] to F'= F[0]
Omega_p[1,0] = Omega_p[1,2] = omega[1]      #σ+ and σ- for F=F[2] to F'=F[0]
Omega_p[2,0] = Omega_p[2,2] = omega[2]     #σ+ and σ- for F=F[3] to F'=F[0]

Delta = [0,10,20.5,20]

H0 = H_0(F,Delta)
HI = H_I(F,Omega_p)
H = 2*np.pi*omega0*a.dag()*a + H0 + HI

#make sure Hamiltonian is sparse
H = H.to("CSR").tidyup(atol=1e-8)

#GammaJgJe = 2.5
GammaJgJe = 2.44745 #1/2π (MHz) 3D1 to 3P0
#GammaJgJe = 20

#single collapse operator for each transtion
c_ops = []
qs = [-1,0,1]

for ig in range(1,len(F)):
    for mfg in range(-F[ig],F[ig] + 1):
        for mfe in range(-F[0], F[0] + 1):
            for q in qs:
                if cg(F,ig,mfg,0,mfe,q) != 0:
                    cops =np.sqrt(1/(2*F[0] + 1))*cg(F,ig,mfg,0,mfe,q)*np.sqrt(GammaFgFe(F,ig,Je,Jg,I,GammaJgJe))*base(F,ig,mfg)*base(F,0,mfe).dag()
                    c_ops.append(cops.to("CSR"))
                else:
                    continue

In [4]:
rho = steadystate(H,c_ops,method='power',solver='spsolve')

In [ ]:
pops = [expect(tensor(basis(Natomic,i),qeye(Nmotion))*tensor(basis(Natomic,i),qeye(Nmotion)).dag(),rho) for i in range(Natomic)] #full steady state

In [6]:
for i in range(len(pops)):
    print(pops[i])

3.267473100341447e-05
3.065199071766561e-05
2.096625275160227e-05
0.0001110723232853451
0.7336239291629619
0.0008130521685430363
0.037799174753872164
0.000506097838219085
0.001788534927362654
0.000779628558618151
0.22421078266214234
0.0002834346305230154


In [7]:
rho_int = rho.ptrace(0)

In [8]:
def base(F,i,mF):
    #numbering states from 0 to N-1 starting from -mF to mF
    # 0 is |F',-mF'>
    if i==0:
        b = basis(Natomic,mF+F[i])
    elif i==1:
        b = basis(Natomic,mF + F[i] + 2*F[0] + 1)
    elif i==2:
        b = basis(Natomic,mF + F[i] + 2*F[0] + 1 + 2*F[1] + 1)
    else:
        b = basis(Natomic,mF + F[i] + 2*F[0] + 1 + 2*F[1] + 1 + 2*F[2] +1)
    return b 

def H1_(F,eta,Omega_p):
    H1=0*basis(Natomic,0)*basis(Natomic,0).dag()
    for mFe in range(-F[0],F[0] + 1):
        for ig in range(1,len(F)):
            for mFg in range(-F[ig],F[ig]+1):
                for q in range(-1,2):
                    H1 += 2*np.pi*1j*eta[ig-1]*cg(F,ig,mFg,0,mFe,q)*Omega_p[ig-1,q+1]/2*base(F,0,mFe)*base(F,ig,mFg).dag()
                
    return -1/np.sqrt(2*F[0]+1)*H1

def H_I(F,Omega_p):
    HI=0*basis(Natomic,0)*basis(Natomic,0).dag()
    for mFe in range(-F[0],F[0] + 1):
        for ig in range(1,len(F)):
            for mFg in range(-F[ig],F[ig]+1):
                for q in range(-1,2):
                    HI += 2*np.pi*cg(F,ig,mFg,0,mFe,q)*Omega_p[ig-1,q+1]/2*base(F,0,mFe)*base(F,ig,mFg).dag()
                
    return -1/np.sqrt(2*F[0]+1)*(HI + HI.dag()) 

def H_0(F,Delta):
    H0 = 0*basis(Natomic,0)*basis(Natomic,0).dag()
    for l in range(len(F)):
        for mF in range(-F[l],F[l]+1):
            H0 += 2*np.pi*(Delta[l] + gF[l]*muB*mF*B)*base(F,l,mF)*base(F,l,mF).dag() 
    return H0

#single collapse operator for each transtion
c_ops = []
qs = [-1,0,1]

for ig in range(1,len(F)):
    for mfg in range(-F[ig],F[ig] + 1):
        for mfe in range(-F[0], F[0] + 1):
            for q in qs:
                if cg(F,ig,mfg,0,mfe,q) != 0:
                    cops =np.sqrt(1/(2*F[0] + 1))*cg(F,ig,mfg,0,mfe,q)*np.sqrt(GammaFgFe(F,ig,Je,Jg,I,GammaJgJe))*base(F,ig,mfg)*base(F,0,mfe).dag()
                    c_ops.append(cops.to("CSR"))
                else:
                    continue

In [9]:
H0_ = H_0(F,Delta)
HI_ = H_I(F,Omega_p) 
    

H0 = H0_ + HI_ 
#H0 = H0.to("CSR").tidyup(atol=1e-8)

c=c_ops

rho0 = steadystate(H0,c)

L0 = 0*spre(c[0])*spost(c[0].dag())
for i in range(len(c)):
    L0 += spre(c[i])*spost(c[i].dag()) - 0.5*(spre(c[i].dag()*c[i]) + spost(c[i].dag()*c[i]))
    #L0 = sum(c)

L0 += -1j*(spre(H0) - spost(H0))
    
#L0 = np.array(L0)
L0 = L0.data_as('ndarray')

H_1 = H1_(F,eta,Omega_p)
L_1 = -1j*(spre(H_1) - spost(H_1))

#L_1 = np.array(L_1)
L_1 = L_1.data_as('ndarray')

H1 = H_1.dag()
L1 = -1j*(spre(H1) - spost(H1))

#L1 = np.array(L1)
L1 = L1.data_as('ndarray')

Deltap=2*np.pi*omega0

N= Natomic
#S3 = -np.matmul(inv(L0-3j*Deltap*np.eye(N**2)),L1)
#S2 = -np.matmul(inv(L0-2j*Deltap*np.eye(N**2)),L1)
#S2 = -np.matmul(inv(L0-2j*Deltap*np.eye(N**2)+ np.matmul(L_1,S3)),L1)
#S1 = -np.matmul(inv(L0-1j*Deltap*np.eye(N**2) + np.matmul(L_1,S2)),L1)
S1 = -np.matmul(inv(L0-1j*Deltap*np.eye(N**2)),L1) 

#T_3 = -np.matmul(inv(L0+3j*Deltap*np.eye(N**2)),L_1)
#T_2 = -np.matmul(inv(L0+2j*Deltap*np.eye(N**2)),L_1)
#T_2 = -np.matmul(inv(L0+2j*Deltap*np.eye(N**2)+ np.matmul(L1,T_3)),L_1)
#T_1 = -np.matmul(inv(L0+1j*Deltap*np.eye(N**2)+ np.matmul(L1,T_2)),L_1)
T_1 = -np.matmul(inv(L0+1j*Deltap*np.eye(N**2)),L_1)
    
L = np.matmul(L_1,S1) + L0 +np.matmul(L1,T_1)


eigenvalues, eigenvectors = eigs(L,k=2,sigma = 0+0j)

rhoss = eigenvectors[:,0]

N=int(np.sqrt(len(rhoss)))

rhos = np.zeros((N,N),dtype='complex64')
for i in range(N):
    for j in range(N):
        rhos[j,i] = rhoss[j+i*N]

In [10]:
eigenvalues

array([ 1.03824590e-16-1.03419025e-16j, -2.17578247e-02-3.98971084e-04j])

In [11]:
rho1 = Qobj(rhos)

In [12]:
rho1=rho1/rho1.tr()

In [13]:
pops = [expect(basis(Natomic,i)*basis(Natomic,i).dag(),rho_int) for i in range(Natomic)]

In [17]:
pops

[3.267473100341447e-05,
 3.065199071766561e-05,
 2.096625275160227e-05,
 0.0001110723232853451,
 0.7336239291629619,
 0.0008130521685430363,
 0.037799174753872164,
 0.000506097838219085,
 0.001788534927362654,
 0.000779628558618151,
 0.22421078266214234,
 0.0002834346305230154]

In [ ]:
pops1 = [expect(basis(Natomic,i)*basis(Natomic,i).dag(),rho1) for i in range(Natomic)] #steady state fictitious lasers

In [ ]:
pops0 = [expect(basis(Natomic,i)*basis(Natomic,i).dag(),rho0) for i in range(Natomic)] #steady state of rest ion

In [20]:
pops1

[(0.006477954524754552+9.276988812562115e-11j),
 (0.005953906040503687+4.9900209398573425e-11j),
 (0.00394138116056771+3.832434803535678e-12j),
 (0.021361258681691288-6.97693708362479e-11j),
 (0.0796923968480801+1.6281435505360875e-09j),
 (0.11156213400399088-2.1859306309490734e-09j),
 (0.12349667391161845+2.990897113308577e-09j),
 (0.09299284782983085-4.168865044840153e-10j),
 (0.2743536688168855+9.530303352045522e-10j),
 (0.1272383085477397-3.3391627587975314e-09j),
 (0.11396410445480987+1.836978416602797e-10j),
 (0.03896536517952753+1.0947788856019436e-10j)]

In [21]:
pops0

[8.16375799890918e-08,
 7.7158600148982e-08,
 5.338997048373485e-08,
 2.8072359329188776e-07,
 0.7691777221433598,
 2.2492195488269402e-06,
 0.00015463468676652972,
 1.2907697923273218e-06,
 4.178034751774213e-06,
 2.0587452637879946e-06,
 0.23065658657092486,
 7.869198486312759e-07]

In [22]:
fidelity(rho1,rho_int)

0.39866546423553395

In [23]:
fidelity(rho0,rho_int)

0.9767601736282865

In [24]:
fidelity(rho0,rho1)

0.2753989428715705

In [25]:
sum(pops1)

(1-3.469446951953614e-18j)

In [ ]:
pop_average = np.array([0.002983774128898129, #average populations during cooling
 0.0018428780072905723,
 0.001609984028225494,
 0.008705012896029585,
 0.32105183613329435,
 0.0519088240468077,
 0.23135013538489693,
 0.047149134383492206,
 0.09826226229560282,
 0.06341279001139988,
 0.15409133077419063,
 0.017632037926755058])

In [27]:
pops=np.array(pops)

In [28]:
pops1 = np.array(pops1).real

In [29]:
pops0 = np.array(pops0).real

In [30]:
from numpy import linalg as LA

In [31]:
LA.norm(pops-pops1)

0.7485101246984004

In [32]:
LA.norm(pops-pops0)

0.05222573460570472

In [33]:
LA.norm(pops-pops)

0.0

In [34]:
LA.norm(pop_average-pops)

0.48062483869252026

In [35]:
LA.norm(pop_average-pops1)

0.3359893194197997

In [36]:
LA.norm(pop_average-pops0)

0.5283225481345616